In [1]:
import os

In [2]:
%pwd

't:\\Projects\\ChickenDisease\\Chicken_Disease_DL\\research'

In [3]:
os.chdir("../")
%pwd

't:\\Projects\\ChickenDisease\\Chicken_Disease_DL'

## Updating the Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:  #dataclass class
    root_dir: Path      #specifies the return type of the class as a collection of all the fields
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


## Updating the Configuration Manager in src config

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        #reading data from yaml files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #create directory at the specified path
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

## Updating the components

In [10]:
import os
import urllib.request as request    #for downloading the dataset
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

# to define the components
class DataIngestion:
    def __init__(self,
                 config: DataIngestionConfig):
        self.config = config

    #funcction to download file from the url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    #function to unzip the data
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Creating the Pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-18 08:02:17,245: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2023-06-18 08:02:17,272: INFO: common: yaml file: params.yaml loaded successfully!]
[2023-06-18 08:02:17,280: INFO: common: created directory at: artifacts]
[2023-06-18 08:02:17,280: INFO: common: created directory at: artifacts/data_ingestion]


[2023-06-18 08:02:32,879: INFO: 2338475769: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9A84:5C50:79B4D4:904487:648E6CB5
Accept-Ranges: bytes
Date: Sun, 18 Jun 2023 02:32:22 GMT
Via: 1.1 varnish
X-Served-By: cache-bur-kbur8200096-BUR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1687055542.932926,VS0,VE864
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 004e70b60c43344bc580843a37e7088431301a71
Expires: Sun, 18 Jun 2023 02:37:22 GMT
Source-Age: 1

]
